In [4]:
import os
import time
import random
import csv
from tqdm.auto import tqdm

import requests
from bs4 import BeautifulSoup

In [5]:
start_page = 1
end_page = 3131 
base_url = 'https://poradna.finance.cz'
csv_filename = '../data/questions_and_answers.csv'
csv_headers = ['Section', 'Theme', 'Question Date', 'Question Title', 'Question Text', 'QA Link', 'Responder Name', 'Answer Date', 'Answer Text']

In [6]:
file_exists = os.path.isfile(csv_filename)
mode = 'a' if file_exists else 'w'
with open(csv_filename, mode, newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    if not file_exists:
        writer.writerow(csv_headers)
    writer.writerow(csv_headers)    
    for page in tqdm(range(start_page, end_page + 1)):
        url = f'https://poradna.finance.cz/?paginator-page={page}&paginator-itemsPerPage=20&do=paginator-paginate'
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error on page {page}: HTTP Status Code {response.status_code}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        next_questions_div = soup.find('div', class_='nextQuestions')
        questions_titles = next_questions_div.find_all('div', class_='responder')
        
        for root_item in questions_titles:
            QA_link = root_item.find_next_sibling('div', class_='dates').find('a', class_='greenText')['href']
            QA_response = requests.get(base_url + QA_link)
            if QA_response.status_code != 200:
                print(f"Error on page {page}: HTTP Status Code {QA_response.status_code}")
                continue
            QA_soup = BeautifulSoup(QA_response.text, 'html.parser')
            QA_div = QA_soup.find('div', class_='consulting')
            question_title = QA_div.find('h1').text.strip().replace('\xa0', ' ')
            section_span = QA_div.find('span')
            section = section_span.find('a').text.strip().replace('\xa0', ' ')
            theme_elements = section_span.find_all('a')
            theme = theme_elements[1].text.strip().replace('\xa0', ' ') if len(theme_elements) > 1 else ""
            
            question_div = QA_soup.find('div', class_='question articleText')
            question_date_span = question_div.find('span', class_='date')
            question_date = question_date_span.text.strip().replace('\xa0', ' ')
            question_text = question_div.find('p').text.strip().replace('\xa0', ' ').replace('\n', ' ')
            
            answer_div = QA_soup.find('div', class_='answer articleText')
            answer_header = answer_div.find('h4')
            answer_date_span = answer_header.find('span', class_='responder')
            responder_text = answer_date_span.text.strip()
            responder_name = responder_text.split(':')[1].split(',')[0].strip().replace('\xa0', ' ')
            answer_date_text = answer_date_span.text.strip()
            answer_date = answer_date_text.split(',')[-1].strip().replace('\xa0', ' ')
            answer_text = answer_div.find('p').text.strip().replace('\xa0', ' ').replace('\n', ' ')
            
            writer.writerow([section, theme, question_date, question_title, question_text, QA_link,responder_name, answer_date, answer_text])  
        time.sleep(random.uniform(0.5, 1.5))

  0%|          | 0/1837 [00:00<?, ?it/s]

Error on page 1443: HTTP Status Code 404
